In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
train_dir = r"e:\Comsats\6th Semester\DL\project\small\train"
test_dir  = r"E:\Comsats\6th Semester\DL\project\small\test"

IMG_SIZE = (128, 128)
BATCH_SIZE = 32
EPOCHS = 20
NUM_CLASSES = 3  # fire, non fire, smoke


In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Image Preprocessing + Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,                # Normalization (0–1)
    rotation_range=20,             # Rotate images
    width_shift_range=0.1,         # Move image left-right
    height_shift_range=0.1,        # Move image up-down
    zoom_range=0.15,               # Zoom in/out
    brightness_range=[0.7, 1.3],   # Change brightness
    horizontal_flip=True,          # Mirror flip
    fill_mode='nearest'            # Fill missing pixels after transformations
)

# Preprocessing for test set (No augmentation)
test_datagen = ImageDataGenerator(
    rescale=1./255                 # Normalization only
)

# Train generator (resizing happens here)
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,          # Resize images to (128,128)
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True
)

# Test generator
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,          # Resize images
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)


Found 1512 images belonging to 3 classes.
Found 610 images belonging to 3 classes.


In [10]:
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128,128,3)),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 126, 126, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 63, 63, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 61, 61, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 30, 30, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 28, 28, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 14, 14, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,211,392 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           387 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,305,027 (12.61 MB)

 Trainable params: 3,305,027 (12.61 MB)

 Non-trainable params: 0 (0.00 B)

In [11]:
from tensorflow.keras.callbacks import EarlyStopping

# Early Stopping to prevent overfitting
early_stop = EarlyStopping(
    monitor='val_loss',        # Stop when validation loss stops improving
    patience=3,                # Wait 3 epochs before stopping
    restore_best_weights=True # Revert to the best weights
)

# Train the model with Early Stopping
history = model.fit(
    train_gen,
    epochs=EPOCHS,
    validation_data=test_gen,
    callbacks=[early_stop],
    verbose=1
)

# Print how many epochs actually ran
print("Training stopped early at epoch:", len(history.history['loss']))


Epoch 1/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 16s 329ms/step - accuracy: 0.6462 - loss: 0.8117 - val_accuracy: 0.7885 - val_loss: 0.5806
Epoch 2/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 315ms/step - accuracy: 0.7778 - loss: 0.6098 - val_accuracy: 0.8246 - val_loss: 0.5091
Epoch 3/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 310ms/step - accuracy: 0.8194 - loss: 0.5196 - val_accuracy: 0.7721 - val_loss: 0.5446
Epoch 4/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 311ms/step - accuracy: 0.8340 - loss: 0.4676 - val_accuracy: 0.7984 - val_loss: 0.5369
Epoch 5/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 320ms/step - accuracy: 0.8393 - loss: 0.4292 - val_accuracy: 0.8508 - val_loss: 0.4227
Epoch 6/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 317ms/step - accuracy: 0.8552 - loss: 0.4073 - val_accuracy: 0.8016 - val_loss: 0.5310
Epoch 7/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 310ms/step - accuracy: 0.8499 - loss: 0.4073 - val_accuracy: 0.8410 - val_loss: 0.4738
Epoch 8/20
48/48 ━━━━━━━━━━━━━━━━━━━━ 15s 308ms/step - accuracy: 0.8466 - loss: 0.4050 - val_accu

In [12]:
loss, acc = model.evaluate(test_gen)
print("Test Accuracy:", round(acc*100, 2), "%")
print("Test Loss:", loss)


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 283ms/step - accuracy: 0.8508 - loss: 0.4227
Test Accuracy: 85.08 %
Test Loss: 0.42265990376472473


In [13]:
# Training Accuracy & Loss
train_acc = history.history['accuracy'][-1]
train_loss = history.history['loss'][-1]

print("Training Accuracy:", round(train_acc * 100, 2), "%")
print("Training Loss:", round(train_loss, 4))

# Validation Accuracy & Loss
val_acc = history.history['val_accuracy'][-1]
val_loss = history.history['val_loss'][-1]

print("Validation Accuracy:", round(val_acc * 100, 2), "%")
print("Validation Loss:", round(val_loss, 4))



Training Accuracy: 84.66 %
Training Loss: 0.405
Validation Accuracy: 76.39 %
Validation Loss: 0.654


#Gradio %

In [ ]:
import gradio as gr
import numpy as np
from tensorflow.keras.models import load_model

model = model  # if same file

# If loading saved model:
# model = load_model("fire_smoke_model.h5")

class_names = ["fire", "non_fire", "smoke"]

def predict_fire(img):
    img = img.resize((128, 128))
    img = np.array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    preds = model.predict(img)[0]

    # gr.Label expects a dict like {"class": probability}
    result = {class_names[i]: float(preds[i]) for i in range(len(class_names))}

    return result

app = gr.Interface(
    fn=predict_fire,
    inputs=gr.Image(type="pil"),
    outputs=gr.Label(num_top_classes=3),
    title="🔥 Fire & Smoke Detection CNN",
    description="Upload an image and the model will detect Fire, Smoke, or Non-Fire."
)

app.launch()


c:\Users\Night-Fury\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "c:\Users\Night-Fury\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Night-Fury\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Night-Fury\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 2191, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Night-Fury\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\blocks.py", line 1702, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\N

In [15]:
import gradio as gr
import numpy as np
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Prediction function
def predict_image(img):
    img = img.resize((128,128))
    img = img_to_array(img) / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)[0]
    classes = ["Fire", "Non Fire", "Smoke"]

    result = {classes[i]: float(pred[i]) for i in range(3)}
    return result

# Custom CSS for Amazing UI
custom_css = """
.gradio-container {background: linear-gradient(135deg,#0F172A,#1E293B);}
h1 {text-align:center; color:white; font-size:38px; margin-bottom:20px;}
#component-0 {padding-top:20px;}
.gr-button {border-radius:12px !important; font-size:18px !important; padding:12px 24px;}
.gr-button-primary {background:#FF5722 !important;}
.gr-input-label, .gr-output-label {color:white !important; font-size:18px;}
"""

demo = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil", label="Upload Image"),
    outputs=gr.Label(num_top_classes=3, label="Prediction"),
    title="🔥 Fire Detection AI System",
    description="Upload an image to classify whether it contains **Fire**, **Smoke**, or **No Fire**.",
    css=custom_css,
    examples=[
        [r"E:\Comsats\6th Semester\DL\project\small\test\fire\Fire (1).jpg"],
        [r"E:\Comsats\6th Semester\DL\project\small\test\non fire\Non_Fire (1).jpg"],
        [r"E:\Comsats\6th Semester\DL\project\small\test\smoke\000072.jpg"]
    ]
)

demo.launch()


* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


#CLASSIFICATION REPORT


In [16]:
from sklearn.metrics import classification_report
import numpy as np

# Get true labels
y_true = test_gen.classes

# Predict probabilities
y_pred_prob = model.predict(test_gen)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Class names
class_labels = list(test_gen.class_indices.keys())

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_labels)
print(report)


20/20 ━━━━━━━━━━━━━━━━━━━━ 6s 287ms/step
              precision    recall  f1-score   support

        fire       0.82      0.92      0.87       202
    non fire       0.87      0.73      0.79       204
       smoke       0.87      0.91      0.89       204

    accuracy                           0.85       610
   macro avg       0.85      0.85      0.85       610
weighted avg       0.85      0.85      0.85       610



In [17]:
#model.save("fire_smoke_model.h5")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
